# Naive Bayes Algorithm

## Classical imports and getting the training dataset

In [1]:
# Necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

Getting the datasets

In [ ]:
tf= pd.read_csv(r'../data/train.csv')
rf= pd.read_csv(r'../data/test.csv')

Checking that the datasets are ok

In [3]:
tf # the training set

,age,gender,scentLover,ecoInterest,washDishes,MrPropre,Antikal,Ariel,Dash,pods,...,likesPets,hasPet,daysSinceActivity,nbChildren,magasin,moyenneSurface,superMarket,hyperMarket,drive,hardDiscount
0,0.077922,1,0.000000,0.387479,1,-1,-1,-1,-1,-1,...,0.9,1,0.234694,0.2,-1,-1,-1,-1,-1,-1
1,0.636364,1,0.378258,0.666667,1,1,1,1,-1,1,...,0.3,-1,0.014914,0.6,-1,-1,-1,-1,-1,-1
2,0.350649,1,0.000000,0.666667,1,-1,-1,1,1,-1,...,0.9,1,0.094976,0.2,-1,-1,-1,-1,-1,-1
3,0.220779,1,0.333333,0.333333,-1,-1,-1,1,1,1,...,0.9,1,0.259812,0.4,-1,-1,-1,-1,-1,-1
4,0.259740,1,0.378258,0.000000,-1,1,1,1,1,-1,...,0.9,1,0.116954,0.0,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0.454545,1,0.378258,0.387479,1,-1,-1,-1,-1,1,...,0.9,1,0.000785,1.0,-1,-1,-1,-1,-1,-1
11996,0.324675,1,0.333333,0.333333,-1,-1,-1,-1,-1,-1,...,0.9,1,0.124019,0.0,-1,-1,-1,-1,-1,-1
11997,0.311688,1,0.378258,0.666667,-1,-1,-1,-1,-1,-1,...,0.9,1,0.018838,0.6,-1,-1,-1,-1,-1,-1
11998,0.272727,-1,0.378258,0.387479,1,-1,-1,-1,-1,-1,...,0.9,1,0.197017,0.2,-1,-1,-1,-1,-1,-1


In [ ]:
rf # the validation set

In [ ]:
#Deleting the Id (we don't want our training to take into account the Id)
tf=tf.drop(columns='userId')
sol=pd.DataFrame({'userId':rf['userId']}) #creating the dataframe for the solution and getting the ID
rf=rf.drop(columns='userId') #dropping the id to have the same structure as the training dataset

In [4]:
# A last clean to have float64 values
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

df=clean_dataset(df)

## Naive Bayes Model

In [5]:
#To see what the best parameters are
def best_model(model):
    print(model.best_score_)    
    print(model.best_params_)
    print(model.best_estimator_)

In [6]:
#splitting our dataset into one set of training and one set of test
y = tf.washDishes.values
x = tf.drop(['washDishes'], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

In [9]:
#Gaussian NB classifier
nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)} ##testing different parameters 

#GridSearch automatically takes the best parameter after testing all the options
gs_NB = GridSearchCV(estimator=nb_classifier, 
                 param_grid=params_NB,
                 verbose=1, 
                 scoring='accuracy') 
gs_NB.fit(x_train, y_train) #the fitting

acc = gs_NB.score(x_test,y_test)*100 #calculating the accuracy

print(best_model(gs_NB)) # to show the best parameters found by GridSearch
print("Test Accuracy {:.2f}%".format(acc)) # to show the accuracy

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.6058781453007357
{'var_smoothing': 0.43287612810830584}
GaussianNB(var_smoothing=0.43287612810830584)
None
Test Accuracy 61.56%


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.5s finished


## Prediction

In [11]:
#Predicting, and exporting to csv to make a submission
y_pred = gs_NB.predict(rf) #the prediction
sol['washDishes'] = y_pred
sol["washDishes"] = sol["washDishes"].apply(lambda e: 'Auto' if e == 1 else 'Hand') #replacing the -1 and 1 with the original values
sol.to_csv('../data/naivebayessolution.csv', index=False) #exporting to csv without the index
sol #checking the output

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.7s finished


,userId,washDishes
0,use_yFJKoOUCJ7ED8c8Xeov1TzdfpRL,Hand
1,use_t1WpMt2fprbjEteXNSZ1TzdyvAi,Hand
2,use_w9jT58WcsP3tUkPCA0A1TzgYdeE,Hand
3,use_onifqa3LnOsjZOhPYfN1TzdfkfA,Hand
4,use_KgP1NlkmR24pbU9KRwK1Tzdqm8u,Hand
...,...,...
3995,use_yXWhQEP4R6vvYOeJzz61TZU7JRs,Hand
3996,use_MafnxsXODJV5oVhfi1h1TzdgYQT,Hand
3997,use_Pbui34WhFhPjNeX6TEz1TzdZdVv,Hand
3998,use_sDxhHoOEzHyrAzWSPnM1TzdYVjB,Hand


## Confusion matrix

In [ ]:
preds=gs_NB.predict(x_test)
def plot_confusion_matrix():
    cm = pd.crosstab(y_test, preds, rownames=['Actual'], colnames=['Predicted'])
    fig, (ax1) = plt.subplots(ncols=1, figsize=(5,5))
    sns.heatmap(cm, 
                xticklabels=['Hand', 'Auto'],
                yticklabels=['Hand', 'Auto'],
                annot=True,ax=ax1,
                linewidths=.2,linecolor="Darkblue", cmap="Blues")
    plt.title('Confusion Matrix', fontsize=14)
    plt.show()

plot_confusion_matrix()